# Model-Assisted (MA) Sampling

This notebook is the supplement to the August 2018 "Biometrics Bits" column in the Forestry Source.

You can run this Python code on your own computer by following the instructions in the README and then running each of the cells in the Jupyter Notebook.

If you have any questions (or just want to geek out about biometrics in general!) contact us at https://silviaterra.com/bark/contact.html

## Overview

This notebook illustrates how model-assisted (MA) sampling works.  We'll work through an example that uses a perfectly rectangular stand that is exactly covered by the remotely-sensed (RS) geotiff located in data/rs-image.tif.  We've cruised 9 plots in this stand and measured the basal area (BA) for each.

We will calculate the mean and standard error of the traditional cruise (no imagery), and then use MA inventory techniques to take advantage of the imagery we have.  We will quantify how much the MA techniques help us improve our standard error and how they adjust our estimate of the mean BA.

In [1]:
# input files are located in the "data" directory

PLOTS_SHP = 'data/plots.shp'  # shapefile with plot locations and basal area
RS_TIF = 'data/rs-image.tif'  # our remote sensing image

In [2]:
# import all the libraries we need

# plotly, including offline setup
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)
from plotly.offline import iplot
import plotly.graph_objs as go

# data analysis
import numpy as np
import scipy.stats as st

# geospatial
import fiona
import rasterio
import pandas as pd
from shapely.geometry import shape

## First, a traditional cruise workup

We'll load the plot data from our shapefile and calculate the mean and standard error - the "old fashioned" way, without imagery.

In [3]:
# load plot data (locations and BA)

plots = []
with fiona.open(PLOTS_SHP) as src:
    for feat in src:
        geom = shape(feat['geometry'])
        plots.append({
            'id': feat['properties']['id'],
            'basal_area': feat['properties']['basal_area'],
            'x': geom.x,
            'y': geom.y
        })

plots_df = pd.DataFrame(plots)

print 'Read in', len(plots_df), 'plots.'

Read in 9 plots.


In [4]:
# display the BA for each plot

plots_df[['id', 'basal_area']]

,id,basal_area
0,1,150
1,2,125
2,3,200
3,4,145
4,5,167
5,6,114
6,7,0
7,8,155
8,9,88


In [5]:
# using traditional statistics, calculate the mean BA and standard error

trad_mean = np.mean(plots_df['basal_area'])
trad_std_err = st.sem(plots_df['basal_area'])

print 'Mean:\t\t %.2f' % trad_mean
print 'Standard Error:\t %.2f' % trad_std_err

Mean:		 127.11
Standard Error:	 19.14


## Now, time to use MA sampling theory to add in some imagery

The main steps here are:
 * Pair up our plots with the RS imagery
 * Build a model that relates the plot BA to the pixel value from the image
 * Apply the model to all pixels to estimate a BA for each pixel
 * Compute the model bias by calculating the average of the residuals
 * Adjust mean BA estimate by subtracting the model bias
 * Compute the standard error by calculating the variance of the residuals

In [6]:
# sample the RS image by plot locations

with rasterio.open(RS_TIF) as src:
    print 'Image dimensions:', src.shape
    pixel_values = [
        x[0] for x in  # have to parse out of returned array
        list(src.sample(plots_df[['x', 'y']].values))
    ]

plots_df['pixel_value'] = pixel_values

Image dimensions: (39, 44)


In [7]:
# Show pixel value vs. BA table

plots_df[['id', 'pixel_value', 'basal_area']]

,id,pixel_value,basal_area
0,1,0.105824,150
1,2,0.110767,125
2,3,0.057252,200
3,4,0.114119,145
4,5,0.105423,167
5,6,0.107453,114
6,7,0.235615,0
7,8,0.089554,155
8,9,0.119050,88


In [8]:
# define a model to predict BA from NDVI

# using a simple linear regression for this example
# in the real world, you'd use something more sophisticated
slope, intercept, r_val, p_val, std_err = st.linregress(plots_df['pixel_value'], plots_df['basal_area'])

def pixel_to_ba(pixel_value):
    # NOTE: added the 1.1 here so that residuals wouldn't sum to zero
    # Don't do this in real life!
    return (slope * 1.1 * pixel_value) + intercept

In [9]:
# graph pixel values vs. BA

# we're using the fantastic Plotly library
# (the graphs only show up if you run this on your computer)
raw_data = go.Scatter(
    name='Raw Data',
    x=plots_df['pixel_value'],
    y=plots_df['basal_area'],
    mode='markers',
    marker={'size': 10}
)

ticks = np.linspace(0, max(plots_df['pixel_value']))
model_trace = go.Scatter(
    name='Model',
    x=ticks,
    y=map(pixel_to_ba, ticks)
)

data = [model_trace, raw_data]

layout = go.Layout(
    xaxis={'title': 'Pixel Value'},
    yaxis={'title': 'Basal Area'}
)

figure = {
    'data': data,
    'layout': layout
}

iplot(figure)

In [10]:
# calculate model residuals

plots_df['predicted_ba'] = map(pixel_to_ba, plots_df['pixel_value'])
plots_df['residual'] = plots_df['predicted_ba'] - plots_df['basal_area']

In [11]:
# show table of model residuals

plots_df[['id', 'basal_area', 'predicted_ba', 'residual']]

,id,basal_area,predicted_ba,residual
0,1,150,126.789587,-23.210413
1,2,125,120.738617,-4.261383
2,3,200,186.252588,-13.747412
3,4,145,116.634477,-28.365523
4,5,167,127.280089,-39.719911
5,6,114,124.794990,10.794990
6,7,0,-32.103667,-32.103667
7,8,155,146.708060,-8.291940
8,9,88,110.597335,22.597335


In [12]:
# predict BA for each pixel

with rasterio.open(RS_TIF) as src:
    pixel_vals = np.ndarray.flatten(src.read(1))
    predicted_bas = map(pixel_to_ba, pixel_vals)

In [ ]:
# write pixel BA out to a geotiff

BA_TIF = 'data/predicted-ba.tif'

with rasterio.open(RS_TIF) as src:
    profile = src.profile
    with rasterio.open(BA_TIF, 'w', **profile) as dst:
        ba_2d = np.reshape(predicted_bas, src.shape)  # turn list into a 2-D matrix
        ba_2d = ba_2d.astype(np.float32)  # avoid dtype error
        dst.write(ba_2d, 1)

In [14]:
# calculate model-assisted mean BA

raw_mean_pix_ba = np.mean(predicted_bas)  # this is the naive mean
mean_residual = np.mean(plots_df['residual'])  # this is the bias in our model
ma_mean = raw_mean_pix_ba - mean_residual  # correct the naive mean by the model bias

print 'Raw mean pixel BA:\t %.2f' % raw_mean_pix_ba
print 'Mean residual:\t\t %.2f' % mean_residual
print 'Model-assisted mean:\t %.2f' % ma_mean

Raw mean pixel BA:	 118.11
Mean residual:		 -12.92
Model-assisted mean:	 131.03


In [15]:
# calculate variance in our model-assisted estimate of the mean

# As Sarndal (1992) notes, the estimate variance equals the variance of the residuals

n = len(plots_df)
ma_var = (1.0 / (n * (n-1))) * sum((plots_df['residual'] - mean_residual)**2)
print 'Model-assisted variance: %.2f' % ma_var

Model-assisted variance: 46.55


In [16]:
# calculate standard error of model-assisted estimate

ma_std_err = np.sqrt(ma_var) / np.sqrt(n)
print 'Model-assisted standard error: %.2f' % ma_std_err

Model-assisted standard error: 2.27


## How did we do?

Let's compare the standard errors that come out of the traditional process and our model-assisted workup.

Note that the MA standard error is signficantly lower and that our estimate of the mean has been slightly adjusted compared to the traditional estimate.  This is the power of adding in auxiliary data (like RS imagery) into your inventory process.

In [17]:
# compare to traditional cruise

comparison_df = pd.DataFrame([
    {'type': 'Traditional', 'mean': trad_mean, 'std_err': trad_std_err},
    {'type': 'Model-Assisted', 'mean': ma_mean, 'std_err': ma_std_err},
])

comparison_df[['type', 'mean', 'std_err']]

,type,mean,std_err
0,Traditional,127.111111,19.144512
1,Model-Assisted,131.029093,2.274273
